# Imports

In [10]:
import pandas as pd
from unidecode import unidecode

# Load Data

In [5]:
df_gupy = pd.read_excel('data_raw/vagas_gupy.xlsx')

df_vagas = pd.concat([df_gupy], axis = 0)
df_vagas.head()

,site_da_vaga,link,data_publicacao,data_coleta,titulo_da_vaga,local,modalidade,contrato,acessibilidade,nome_empresa,codigo_vaga,descricao
0,Gupy,https://globo.gupy.io/job/eyJqb2JJZCI6NjYzMjY2...,Publicada em: 16/01/2024,2024-01-17,Analista de dados Júnior - Prazo Determinado |...,Rio de Jan... - RJ,Híbrido,Temporário,NaN,Globo,NaN,Descrição da vagaNossa área de Estratégia e Te...
1,Gupy,https://magnasistemas.gupy.io/job/eyJqb2JJZCI6...,Publicada em: 16/01/2024,2024-01-17,6593545| Analista de Dados PL,São Paulo - SP,Presencial,Efetivo,Também p/ PcD,MAGNA SISTEMAS,NaN,Descrição da vagaJunte-se a nós!▶️Que tal atua...
2,Gupy,https://recrutamentoescalar.gupy.io/job/eyJqb2...,Publicada em: 16/01/2024,2024-01-17,Analista de Inteligência de Dados,Belo Horiz... - MG,Híbrido,Efetivo,NaN,Recrutamento e Seleção de Profissionais Grupo ...,NaN,Descrição da vagaSobre a empresaEmpresa dedica...
3,Gupy,https://magnasistemas.gupy.io/job/eyJqb2JJZCI6...,Publicada em: 16/01/2024,2024-01-17,6597603| Analista de Ciências de Dados PL,Taboão da ... - SP,Híbrido,Efetivo,Também p/ PcD,MAGNA SISTEMAS,NaN,Descrição da vagaJunte-se a nós!▶️Que tal atua...
4,Gupy,https://nava.gupy.io/job/eyJqb2JJZCI6NjYyOTk0N...,Publicada em: 16/01/2024,2024-01-17,Analista de Automação (Banco de Dados) PL/SR,São Paulo - SP,Híbrido,Efetivo,Também p/ PcD,NAVA | Technology for Business,NaN,Descrição da vagaA NAVA está em busca de uma P...


# Features

## Senioridade

In [ ]:
# matches para as senioridades
junior_matches = ['junior', ' jr', ' i', ' l']
pleno_matches = ['pleno', ' pl', ' ii', ' ll']
senior_matches = ['senior', ' sr', ' iii', 'lll']

# removendo acentos e deixando letras minúsculas
df_vagas['titulo_da_vaga_tratado'] = df_vagas['titulo_da_vaga'].apply(lambda x: unidecode(x.lower()))

# função de faz matches com as senioridades das vagas
def busca_senioridade(titulo_vaga, match_list):
    senioridade = False
    for sub_string in match_list:
        if sub_string in titulo_vaga:
            senioridade = True
    
    return senioridade

# aplicando função
df_vagas['senioridade_junior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, junior_matches))
df_vagas['senioridade_pleno'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, pleno_matches))
df_vagas['senioridade_senior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, senior_matches))

# categorizando senioridades
df_vagas['senioridade'] = df_vagas[['senioridade_junior', 
                                    'senioridade_pleno', 
                                    'senioridade_senior']].apply(lambda x:  'Pleno/Sênior' if (x['senioridade_pleno']) and (x['senioridade_senior']) else
                                                                            'Júnior/Pleno' if (x['senioridade_junior']) and (x['senioridade_pleno']) else
                                                                            'Júnior' if x['senioridade_junior'] else 
                                                                            'Pleno' if x['senioridade_pleno'] else
                                                                            'Sênior' if x['senioridade_senior'] else
                                                                            'Não informado'
                                                                            , axis = 1)

In [97]:
df_vagas.loc[df_vagas['senioridade'] == 'Não informado', 'titulo_da_vaga_tratado'].unique()

array(['pessoa analista de dados - geoprocessamento | manufacturing.',
       'analista de dados (remoto)', 'analista de dados | salvador',
       'analista de dados (tableau)', 'analista de dados | mercado ',
       'analista de dados de transporte', 'analista de dados',
       'analista de privacidade de dados (lgpd) - presencial brasilia',
       'analista de customer service - dados | ribas do rio pardo/ms',
       'logistica: analista de dados', 'analista de dados - jundiai/sp ',
       'analista de dados comerciais', 'analista dados ti',
       'analista de dados comerciais - direcionado a vendas',
       'analista contabil - analista de dados ',
       'analista de marketing - crm (dados)',
       'cas | analista de dados - foco em associacao e contas',
       'analista de dados | cresol confederacao',
       'pessoa analista de dados (cs/cx)', 'analista de dados csc',
       'analista de dados | data analyst',
       'analista de dados ( pricing) ',
       'pessoa analista de b

# Save Dataframe

In [103]:
columns_selected = [
'site_da_vaga',
 'link',
 'data_publicacao',
 'data_coleta',
 'titulo_da_vaga',
 'senioridade',
 'local',
 'modalidade',
 'contrato',
 'acessibilidade',
 'nome_empresa',
 'codigo_vaga',
 'descricao'
]

In [104]:
df_vagas[columns_selected].to_excel('vagas_clean.xlsx', index=False)